In [ ]:
import requests
from bs4 import BeautifulSoup

#10개 페이지 넘기면서 크롤링!
for page_num in range(3):
        res = requests.get(f"https://www.yeogi.com/domestic-accommodations?keyword=%EC%84%9C%EC%9A%B8&checkIn=2025-09-11&checkOut=2025-09-12&personal=2&freeForm=false&page={str(page_num+1)}") # 문자열의 형태로 바꿔서 안젼하게!
        
    soup = BeautifulSoup(res.content,"html.parser") # 파싱을 통해 사람이 알 수 있는 값으로 출력해줌
    #print(soup)
    data = soup.select("div.card-body") #card -body안에 있는 모든 데이터를 가져옴.
    #print(data)

    #mongo db 에 저장할때 나눠서 저장하기 위해 data의 값들을 나눠줘야함 -> for in 문
    for item in data:
        category = item.select_one("h2.card-title").get_text().replace("관련 상품 추천","").strip() 
        # h2태그의 card-title을 찾아옴 # 코드는 빼고 타이틀만 찾아오기
        product = item.select_one("h4.card-text").get_text().replace("관련 상품 추천","").strip() 
        print(category,product) 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import urllib
import time

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

query = "#sun"
encoded_query = urllib.parse.quote(query)

# url = f"https://brunch.co.kr/serial/list{query}#PUBLISH_TIME"
# driver.get(url)
# time.sleep(1)
weeks = ["#mon", "#tue", "#wed", "#thu", "#fri", "#sat", "#sun"]
# titles= driver.find_elements(By.CSS_SELECTOR, "strong.tit_info")

for week in weeks :
    driver.get(f"https://brunch.co.kr/serial/list{week}#PUBLISH_TIME")
    time.sleep(1)
    print(f"\n{week} 베스트\n")
    titles= driver.find_elements(By.CSS_SELECTOR, "strong.tit_info")
    for i, title in enumerate(titles, 1) :
        print(f"{i}. {title.text.strip().replace("\n","")}")

driver.quit()

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# 셀레니움 드라이버 설정
service = Service(ChromeDriverManager().install())
options = Options()
options = Options()
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu") # GPU 렌더링 비활성화(옛날 headless 버그 회피).
options.add_argument("--disable-dev-shm-usage") # 리뷰가 많을 때 임시메모리에 저장 못함. 우리 로컬컴퓨터의 메모리를 사용할 수 있게 해주는것.
options.add_argument("--start-maximized")  # 이게 있어야 반응형 페이지로 잡힘(?)
options.add_argument("--window-size=1920x1080")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

# 크롤링할 URL (서울, 2인, 1박 기준)
query = 1
url = f"https://www.yeogi.com/domestic-accommodations?keyword=%EC%84%9C%EC%9A%B8&checkIn=2025-09-11&checkOut=2025-09-12&personal=2&freeForm=false&page={query+1}"
driver.get(url)
time.sleep(3)  # JavaScript 로딩 대기
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 숙소 정보 추출
hotels = driver.find_elements(By.CSS_SELECTOR, "div.css-1g90zg3")

print("\n📍 [여기어때 - 서울 숙소 검색 결과]")

for i, hotel in enumerate(hotels, 1):
    try:
        name = hotel.find_element(By.CSS_SELECTOR, "h3.gc-thumbnail-type-seller-card-title.css-bm6g3w").text.strip()
    except:
        name = "(숙소명 없음)"
    
    try:
        price = hotel.find_element(By.CSS_SELECTOR, "span.css-1qdxuc0").text.strip()
    except:
        price = "(가격 정보 없음)"
    
    try:
        review = hotel.find_element(By.CSS_SELECTOR, "span.css-9ml4lz").text.strip()
    except:
        review = "(리뷰 없음)"

    print(f"{i}. {name} | 💰 {price} | 📝 {review}")

driver.quit()


📍 [여기어때 - 서울 숙소 검색 결과]
